<a href="https://colab.research.google.com/github/BurakKizilkaya/Machine-Learning/blob/master/KIZILKAYA_aerial_cactus_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNG 562 - HOMEWORK 2

# Aerial Cactus Identification








## Convolution Neural Networks(CNN)

In [1]:
#importing kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"burakkizilkaya","key":"7c7b6f9585e0232bc29c2fa80e888786"}'}

In [2]:
#if there is no kaggle directory, create a new one.
!mkdir -p ~/.kaggle
#copy kaggle.json file to kaggle directory
!cp kaggle.json ~/.kaggle/
#give owner read and write access permissions
!chmod 600 ~/.kaggle/kaggle.json
#check files in kaggle directory
!ls ~/.kaggle

kaggle.json


In [3]:
#install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 5.3MB 14.8MB/s 
     |████████████████████████████████| 112kB 50.1MB/s 
     |████████████████████████████████| 102kB 29.6MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 


In [4]:
#download cactus-aerial-photos dataset
!kaggle datasets download -d irvingvasquez/cactus-aerial-photos

100% 47.3M/47.3M [00:00<00:00, 46.9MB/s]



In [5]:
#unzip downloaded dataset file
!unzip cactus-aerial-photos.zip


Archive:  cactus-aerial-photos.zip
  inflating: training_set.zip        
  inflating: validation_set.zip      


In [0]:
#unzip downloaded train and validation datasets
!unzip training_set.zip
!unzip validation_set.zip

In [7]:
# Using Keras Sequential Model

#import libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

cnn = Sequential()

Using TensorFlow backend.


In [8]:
#Adding 1st Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3),input_shape = (64, 64, 3), activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Adding 2nd Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3),activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Adding 3rd Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3), activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Adding 4th Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3), activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Flattening
cnn.add(Flatten())

cnn.add(Dense(units=128,activation='relu'))
cnn.add(Dense(units = 1, activation = 'sigmoid'))

#compile the model
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# data generation from cactus images
traindatagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
testdatagen = ImageDataGenerator(rescale=1./255)
train = traindatagen.flow_from_directory('training_set', target_size=(64,64), batch_size=32, class_mode = 'binary')
test = testdatagen.flow_from_directory('validation_set', target_size=(64,64), batch_size=32, class_mode = 'binary')

Found 17500 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [10]:
# train the model on data 
acc = cnn.fit_generator(train, steps_per_epoch = 100, epochs = 30, validation_data = test, validation_steps = 50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
100/100 [==============================] - 29s 292ms/step - loss: 0.3912 - acc: 0.8206 - val_loss: 0.2353 - val_acc: 0.9169
Epoch 2/30
100/100 [==============================] - 28s 283ms/step - loss: 0.2281 - acc: 0.9206 - val_loss: 0.2057 - val_acc: 0.9275
Epoch 3/30
100/100 [==============================] - 28s 281ms/step - loss: 0.1931 - acc: 0.9231 - val_loss: 0.2017 - val_acc: 0.9163
Epoch 4/30
100/100 [==============================] - 28s 282ms/step - loss: 0.1597 - acc: 0.9394 - val_loss: 0.2098 - val_acc: 0.9187
Epoch 5/30
100/100 [==============================] - 28s 282ms/step - loss: 0.1471 - acc: 0.9416 - val_loss: 0.1937 - val_acc: 0.9356
Epoch 6/30
100/100 [==============================] - 28s 284ms/step - loss: 0.1386 - acc: 0.9496 - val_loss: 0.1475 - val_acc: 0.9413
Epoch 7/30
100/100 [==============================] - 28s 281ms/step - loss: 0.1167 - acc: 0.9550 - val_loss: 0.1385 - val_acc: 0.9500
Epoch 8

# Transfer Learning

In [15]:
# RESNET50 from keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.python.keras import optimizers
import numpy as np

model = Sequential()

model.add(ResNet50(weights='imagenet'))
model.add(Dense(1, activation = 'relu'))

# dont train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

# data generation from cactus images
traindatagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
testdatagen = ImageDataGenerator(rescale=1./255)
train = traindatagen.flow_from_directory('training_set', target_size=(224,224), batch_size=32, class_mode = 'binary')
test = testdatagen.flow_from_directory('validation_set', target_size=(224,224), batch_size=32, class_mode = 'binary')


model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



Found 17500 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [16]:
# epoch number can be increased to 30. It is kept as 1 since it takes too much time
acc = model.fit_generator(train, steps_per_epoch = 100, epochs = 1, validation_data = test, validation_steps = 50)


Epoch 1/1
100/100 [==============================] - 1447s 14s/step - loss: 0.9256 - acc: 0.7534 - val_loss: 0.7526 - val_acc: 0.7481
